In [9]:
import pylab as pb
import GPy
import numpy as np

pb.interactive(False)

In [10]:
#This functions generate data corresponding to two outputs
f_output1 = lambda x: 4. * np.cos(x/5.) - .4*x - 35. + np.random.rand(x.size)[:,None] * 2.
f_output2 = lambda x: 6. * np.cos(x/5.) + .2*x + 35. + np.random.rand(x.size)[:,None] * 8.


#{X,Y} training set for each output
X1 = np.random.rand(100)[:,None]; X1=X1*75
X2 = np.random.rand(100)[:,None]; X2=X2*70 + 30
Y1 = f_output1(X1)
Y2 = f_output2(X2)
#{X,Y} test set for each output
Xt1 = np.random.rand(100)[:,None]*100
Xt2 = np.random.rand(100)[:,None]*100
Yt1 = f_output1(Xt1)
Yt2 = f_output2(Xt2)

In [11]:
K = GPy.kern.Matern32(1)
icm = GPy.util.multioutput.ICM(input_dim=1, num_outputs=2, kernel=K)
m = GPy.models.GPCoregionalizedRegression([X1, X2], [Y1, Y2], kernel=icm)
#For this kernel, B.kappa encodes the variance now.
m['.*Mat32.var'].constrain_fixed(1.) 
m.optimize()
print(m)


Name : gp
Objective : 373.08077792479105
Number of Parameters : 8
Number of Optimization Parameters : 7
Updates : True
Parameters:
  gp.                                    |               value  |  constraints  |  priors
  ICM.Mat32.variance                     |                 1.0  |   +ve fixed   |        
  ICM.Mat32.lengthscale                  |   1211.397804244578  |      +ve      |        
  ICM.B.W                                |              (2, 1)  |               |        
  ICM.B.kappa                            |                (2,)  |      +ve      |        
  mixed_noise.Gaussian_noise_0.variance  |  0.3238206191950899  |      +ve      |        
  mixed_noise.Gaussian_noise_1.variance  |   4.970105724250054  |      +ve      |        


 /home/saumya/miniconda3/envs/tf/lib/python3.9/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in divide
 /home/saumya/miniconda3/envs/tf/lib/python3.9/site-packages/GPy/kern/src/stationary.py:489: RuntimeWarning:overflow encountered in multiply
 /home/saumya/miniconda3/envs/tf/lib/python3.9/site-packages/GPy/kern/src/stationary.py:489: RuntimeWarning:invalid value encountered in multiply
 /home/saumya/miniconda3/envs/tf/lib/python3.9/site-packages/GPy/kern/src/stationary.py:492: RuntimeWarning:overflow encountered in multiply
 /home/saumya/miniconda3/envs/tf/lib/python3.9/site-packages/GPy/kern/src/stationary.py:492: RuntimeWarning:invalid value encountered in multiply


In [12]:
kernel = GPy.util.multioutput.ICM(input_dim=1,num_outputs=2,kernel=GPy.kern.Bias(input_dim=1))
m = GPy.models.GPCoregionalizedRegression(X_list=[X1,X2],Y_list=[Y1,Y2],kernel=kernel)
#B.kappa now encodes the variance.
m['.*bias.var'].constrain_fixed(1) 
m['.*W'].constrain_fixed(0)
m.optimize()

Alt 

In [13]:
newX = np.arange(100,110)[:,None]
newX = np.hstack([newX,np.ones_like(newX)])
print(newX)

[[100   1]
 [101   1]
 [102   1]
 [103   1]
 [104   1]
 [105   1]
 [106   1]
 [107   1]
 [108   1]
 [109   1]]


In [14]:
noise_dict = {'output_index':newX[:,1:].astype(int)}

In [15]:
m.predict(newX,Y_metadata=noise_dict)

(array([[64.99408995],
        [65.34465319],
        [65.68768604],
        [66.0232731 ],
        [66.35149819],
        [66.67244434],
        [66.98619382],
        [67.29282809],
        [67.59242789],
        [67.88507315]]),
 array([[ 7.33593179],
        [ 9.24360347],
        [12.21572235],
        [16.47005211],
        [22.21513687],
        [29.6505505 ],
        [38.96714038],
        [50.34726559],
        [63.96502993],
        [79.98650959]]))